# import statements

In [32]:
import numpy as np
import pandas as pd
from glob import glob
import librosa
from sklearn.decomposition import PCA


# For preprocessing
import wave
import os

# for avoiding bad audio files
from soundfile import LibsndfileError, SoundFileRuntimeError

# Loading in raw input data

In [15]:
# Load in the data from the specified directory
raw_train_files = glob('train/*')

# Load the audio file using librosa
# y = audio time series

# Method for splits valid data files
def split_data(input_dir, output_dir, segment_duration):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.wav'):
                input_file = os.path.join(root, file)
                try:
                    with wave.open(input_file, 'rb') as wf:
                        framerate = wf.getframerate()
                        nframes = wf.getnframes()
                        # Gets the full length of the audio in seconds
                        duration = nframes / framerate

                        # Calc frames per seg
                        frames_per_seg = int(segment_duration * framerate)
                        num_segments = 26

                        # split the file into segments
                        for i in range(num_segments):
                            start_frame = i * frames_per_seg
                            end_frame = (i + 1) * frames_per_seg
                            
                            # set position at time we want
                            wf.setpos(i * framerate)
                            # Read audio data for this segment
                            segment_data = wf.readframes(frames_per_seg)

                            # Write the segment to a new file
                            output_file = os.path.join(output_dir, f'{os.path.splitext(os.path.basename(input_file))[0]}_segment_{i}.wav')
                            with wave.open(output_file, 'wb') as segment_wf:
                                segment_wf.setparams(wf.getparams())
                                segment_wf.writeframes(segment_data)
                except Exception as e:
                    print(f"Error splitting audio file {input_file}: {e}")
    return output_dir

train_files_dir = split_data("train", "split_output", 5)
train_files = glob(train_files_dir + "/*")
print(len(train_files))
time_series = []
#load the files from the directory
for filename in train_files:
    try:
        y, _ = librosa.load(filename)
        # print("Iteration " + str(i) + " success")
        time_series.append(y)
    except Exception as e:
        print(f"Error loading audio file {filename}: {e}")




20957


# Basic feature extraction

In [19]:
def extract_features(series):
    """
    Uses Librosa to extract features from the time series.
    series: list of floats
    returns:
    spectral_centroid: the center of mass of the spectrum
    spectral rolloff: the frequency below which 85% of the magnitude distribution is concentrated
    spectral bandwidth: the width of the band of frequencies
    spectral contrast: the difference in amplitude between peaks and valleys in the spectrum
    spectral flatness: the flatness of a signal
    spectral rms: the root mean square of the signal
    """
    sr = 22050
    spectral_centroid = librosa.feature.spectral_centroid(y=series)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=series)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=series)
    spectral_contrast = librosa.feature.spectral_contrast(y=series)
    spectral_flatness = librosa.feature.spectral_flatness(y=series)
    rms = librosa.feature.rms(y=series)
    onset_env = librosa.onset.onset_strength(y=series, sr=sr)
    tempo, beats = librosa.beat.beat_track(y =series, sr=sr)
    beat_strengths = onset_env[beats]
    key = librosa.feature.chroma_stft(y=series, sr=sr)

    return spectral_centroid, spectral_rolloff, spectral_bandwidth, spectral_contrast, spectral_flatness, rms, tempo, beat_strengths, key

# centroids = []
# rolloffs = []
# bandwidths = []
# contrasts = []
# flatnesses = []
# rms = []
# tempos = []
# beat_strengths = []
# keys = []


# for i in range(0, len(time_series)):
#     spectral_centroid, spectral_rolloff, spectral_bandwidth, spectral_contrast, spectral_flatness, spectral_rms, tempo, beat_strength, key = extract_features(time_series[i])
#     centroids.append(spectral_centroid)
#     rolloffs.append(spectral_rolloff)
#     bandwidths.append(spectral_bandwidth)
#     contrasts.append(spectral_contrast)
#     flatnesses.append(spectral_flatness)
#     rms.append(spectral_rms)
#     tempos.append(tempo)
#     beat_strengths.append(beat_strength)
#     keys.append(key)




In [20]:
def extract_MORE_features(series):
    """
    Uses Librosa to extract features from the time series.
    series: list of floats
    returns:
    zero_crossing_rate: the rate of sign changes in the signal
    mfcc: Mel-frequency cepstral coefficients
    """
    zero_rate = librosa.feature.zero_crossing_rate(y=series)
    mfcc = librosa.feature.mfcc(y=series)


    return zero_rate, mfcc

# zero_rates = []
# mfccs = []

# for i in range(0, len(time_series)):
#     zero_rate, mfcc = extract_MORE_features(time_series[i])
#     zero_rates.append(zero_rate)
#     mfccs.append(mfcc)


In [ ]:
# Collect Features into data matrix ( we are only using some from before, but you can use whichever ones you want)
spectral_centroid, spectral_rolloff, spectral_bandwidth, spectral_contrast, spectral_flatness, spectral_rms, tempo, beat_strength, key = extract_features(time_series[0])
zero_rate, mfcc = extract_MORE_features(time_series[0])
feature = np.concatenate((np.ravel(spectral_centroid), np.ravel(spectral_rolloff), np.ravel(spectral_bandwidth), np.ravel(spectral_contrast), np.ravel(spectral_flatness), np.ravel(spectral_rms), np.ravel(tempo), np.ravel(beat_strength), np.ravel(key), np.ravel(zero_rate), np.ravel(mfcc)))
for i in range(1, len(time_series)):
    spectral_centroid, spectral_rolloff, spectral_bandwidth, spectral_contrast, spectral_flatness, spectral_rms, tempo, beat_strength, key = extract_features(time_series[i])
    zero_rate, mfcc = extract_MORE_features(time_series[i])
    feature_to_add = np.concatenate((np.ravel(spectral_centroid), np.ravel(spectral_rolloff), np.ravel(spectral_bandwidth), np.ravel(spectral_contrast), np.ravel(spectral_flatness), np.ravel(spectral_rms), np.ravel(tempo), np.ravel(beat_strength), np.ravel(key), np.ravel(zero_rate), np.ravel(mfcc)))
    feature = np.hstack((feature, feature_to_add))


In [ ]:
pca = PCA(n_components = 100)
new_feature = pca.fit_transform(feature)

In [47]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Conv2D, MaxPooling2D, Flatten
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from scipy.stats import randint

In [44]:
data = []
for i in range(0, len(time_series)):
    data.append(librosa.feature.melspectrogram(y=time_series[i], sr=22050))

    

In [55]:
model = Sequential()

model.add(Conv2D(5, (5, 5), activation='relu', input_shape=(128, 216, 1)))
model.add(MaxPooling2D((3, 3)))
model.add(Conv2D(7, (5, 5), activation='relu'))
model.add(MaxPooling2D((3, 3)))
model.add(Conv2D(7, (5, 5), activation='relu'))
model.add(MaxPooling2D((3, 3)))
model.add(Flatten())

model.add(Dense(10, activation='relu', input_shape = (65,), kernel_regularizer='l2', bias_regularizer='l2', activity_regularizer='l2'))

model.add(Dense(256, activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(60, activation='softmax'))

model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# model.fit(features_train, y_train, epochs=25, batch_size=28, validation_split=0.2, verbose=2, shuffle=True, steps_per_epoch=10, validation_steps=10, validation_batch_size=28, validation_freq=1)

# loss, accuracy = model.evaluate(features_train, y_train)
print('Neural Network accuracy:', accuracy)

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_25 (Conv2D)              │ (None, 124, 212, 5)    │           130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 41, 70, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 37, 66, 7)      │           882 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 12, 22, 7)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 8, 18, 7)       │         1,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 2, 6, 7)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 84)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 10)             │           850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 256)            │         2,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 60)             │         7,740 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,546 (181.82 KB)

 Trainable params: 46,546 (181.82 KB)

 Non-trainable params: 0 (0.00 B)

NameError: name 'accuracy' is not defined

# Export the Feature data

In [33]:
def aggregate_features(features):
    """
    stores the mean, standard deviation, max, and min of the features
    """
    means = []
    stds = []
    maxs = []
    mins = []
    for feature in features:
        means.append(np.mean(feature))
        stds.append(np.std(feature))
        maxs.append(np.max(feature))
        mins.append(np.min(feature))
    return means, stds, maxs, mins

def save_features(features, filename):
    """
    saves the features to a csv file
    """
    df = pd.DataFrame({filename: features})
    df.to_csv('features_test/' + filename + '.csv', index=False)

In [17]:
aggregates = []
for feature in features:
    mean, std, max, min = aggregate_features(feature)
    aggregates.append(mean)
    aggregates.append(std)
    aggregates.append(max)
    aggregates.append(min)
aggregates.append(tempos)
feature_count = 0
for aggregate in aggregates:
    save_features(aggregate, 'feature_' + str(feature_count))
    feature_count += 1



In [25]:
def evenMOREfeatures(series):
    """
    Uses Librosa to extract features from the time series.
    series: list of floats
    returns:
    chroma: the chroma of the signal
    """
    tonnetz = librosa.feature.tonnetz(y=series)
    chroma = librosa.feature.chroma_stft(y=series)
    harmonic, percussive = librosa.effects.hpss(y=series)
    harmonic_chroma = librosa.feature.chroma_cqt(y=harmonic)
    percussive_tempo, _ = librosa.beat.beat_track(y=percussive)

    return tonnetz, chroma, harmonic_chroma, percussive_tempo

tonnetzs = []
chromas = []
harmonic_chromas = []
percussive_tempos = []

for i in range(0, len(time_series)):
    tonnetz, chroma, harmonic_chroma, percussive_tempo = evenMOREfeatures(time_series[i])
    tonnetzs.append(tonnetz)
    chromas.append(chroma)
    harmonic_chromas.append(harmonic_chroma)
    percussive_tempos.append(percussive_tempo)


In [26]:
aggregate_more = []
for feature in [tonnetzs, chromas, harmonic_chromas, percussive_tempos]:
    mean, std, max, min = aggregate_features(feature)
    aggregate_more.append(mean)
    aggregate_more.append(std)
    aggregate_more.append(max)
    aggregate_more.append(min)
    
for aggregate in aggregate_more:
    save_features(aggregate, 'feature_' + str(i))
    feature_count += 1


In [34]:
def more_features(series):
    """
    Uses Librosa to extract features from the time series.
    series: list of floats
    returns:
    mfcc_delta: the change in mfcc
    """
    mfcc_delta = librosa.feature.delta(librosa.feature.mfcc(y=series))

    return mfcc_delta

mfcc_deltas = []

for i in range(0, len(time_series)):
    mfcc_delta = more_features(time_series[i])
    mfcc_deltas.append(mfcc_delta)

aggregate_mfcc_delta = []
for feature in [mfcc_deltas]:
    mean, std, max, min = aggregate_features(feature)
    aggregate_mfcc_delta.append(mean)
    aggregate_mfcc_delta.append(std)
    aggregate_mfcc_delta.append(max)
    aggregate_mfcc_delta.append(min)
feature_count = 61
for aggregate in aggregate_mfcc_delta:
    save_features(aggregate, 'feature_' + str(feature_count))
    feature_count += 1